In [1]:
!pip install opencv-python
!pip install ultralytics

In [2]:
import os
import cv2
from ultralytics import YOLO
import torch


In [3]:
if torch.cuda.is_available():
    print("CUDA khả dụng. Đang sử dụng GPU.")
else:
    print("CUDA không khả dụng. Đang sử dụng CPU.")

CUDA khả dụng. Đang sử dụng GPU.


In [4]:
def calculate_iou(box1, box2):
    box1 = torch.tensor(box1)
    box2 = torch.tensor(box2)
    x1 = torch.max(box1[0], box2[0])
    y1 = torch.max(box1[1], box2[1])
    x2 = torch.min(box1[2], box2[2])
    y2 = torch.min(box1[3], box2[3])

    # Diện tích phần giao nhau
    intersection_area = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)

    # Diện tích bounding box
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Tính IoU
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou.item()
def filter_overlapping_boxes(bboxes_with_conf, overlap_threshold=0.3):
    bboxes_with_conf.sort(key=lambda x: x[4], reverse=True)  # Sắp xếp theo conf giảm dần
    filtered_bboxes = []

    for i, box1 in enumerate(bboxes_with_conf):
        keep = True
        for box2 in filtered_bboxes:
            overlap = calculate_iou(box1[:4], box2[:4])
            if overlap > overlap_threshold:
                keep = False
                break
        if keep:
            filtered_bboxes.append(box1)

    return filtered_bboxes

def check_glasses_position(glasses_box, keypoints):
    glasses_center_x = (glasses_box[0] + glasses_box[2]) / 2
    glasses_center_y = (glasses_box[1] + glasses_box[3]) / 2

    # Lấy tọa độ các keypoint
    nose_x, nose_y = keypoints[0]  # Mũi
    left_eye_x, left_eye_y = keypoints[1]  # Mắt trái
    right_eye_x, right_eye_y = keypoints[2]  # Mắt phải
    left_shoulder_x, left_shoulder_y = keypoints[5]  # Vai trái
    right_shoulder_x, right_shoulder_y = keypoints[6]  # Vai phải

    # Kiểm tra kính trên túi áo
    if (left_shoulder_y < glasses_center_y and
        left_shoulder_x > glasses_center_x > right_shoulder_x):
        return "Pocket"

    # Kiểm tra kính trên đầu
    head_top_y = (left_eye_y + right_eye_y) / 2 - 0.2 * abs(left_eye_y - right_eye_y)
    if glasses_center_y < head_top_y:
        return "Head"

    return "Wearing"

In [8]:
image_folder = 'Glasses'
output_folder = 'final_output'
os.makedirs(output_folder, exist_ok=True)

# Load mô hình
yolo_model = YOLO('yolov8x-world.pt')
yolo_model.set_classes(["glasses", "sunglasses"])
pose_model = YOLO('yolov8n-pose.pt')

# Lặp qua từng ảnh
for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, filename)
        # Bước 1: Dò kính (bounding boxes cho kính)
        glass_results = yolo_model.predict(image_path)
        glasses_bboxes_with_conf = [
            box[:6] for box in glass_results[0].boxes.data.tolist()
        ]  # Tọa độ kính, độ tin cậy (conf), và class
        
        # Lọc các bb kính bị trùng lặp
        glasses_bboxes_with_conf = filter_overlapping_boxes(glasses_bboxes_with_conf)
        
        # Bước 2: Dò người (bounding boxes và keypoints)
        pose_results = pose_model.predict(image_path, conf=0.5)
        people_bboxes = [box[:4] for box in pose_results[0].boxes.data.tolist()]
        people_keypoints = pose_results[0].keypoints.xy if pose_results[0].keypoints is not None else []
        
        # Liên kết kính với người dựa trên IoU
        glasses_to_people = []
        for glass_box_with_conf in glasses_bboxes_with_conf:
            glass_box = glass_box_with_conf[:4]
            max_iou = 0
            assigned_person = None
            for person_idx, person_box in enumerate(people_bboxes):
                iou = calculate_iou(glass_box, person_box)
                if iou > max_iou:
                    max_iou = iou
                    assigned_person = person_idx
        
            # Đánh dấu kính với người tương ứng (nếu có)
            if assigned_person is not None:
                glasses_to_people.append((glass_box_with_conf, assigned_person))
        
        # Bước 3: Kiểm tra vị trí kính
        annotated_image = cv2.imread(image_path)
        height, width, _ = annotated_image.shape
        
        font_scale = min(width, height) / 1000  # Tỉ lệ chữ dựa trên kích thước ảnh
        font_thickness = max(1, int(font_scale * 2.5))  # Độ dày phông chữ
        
        for glass_box_with_conf, person_idx in glasses_to_people:
            glass_box = glass_box_with_conf[:4]
            conf = glass_box_with_conf[4]  # Độ tin cậy
            cls = "glasses" if int(glass_box_with_conf[5]) == 0 else "sunglasses"  # Class
            keypoints = people_keypoints[person_idx]
            position = check_glasses_position(glass_box, keypoints)
        
            # Annotate ảnh với vị trí kính, độ tin cậy và class
            x1, y1, x2, y2 = map(int, glass_box)
            text = f"{position}-{cls}-Conf:{conf:.2f}"
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_image, text, (x1, y2 + 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
        
        # Lưu ảnh kết quả
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, annotated_image)



image 1/1 /home/phong/PycharmProjects/Glasses_Detetor/Glasses/5.png: 448x640 1 glasses, 59.1ms
Speed: 4.4ms preprocess, 59.1ms inference, 35.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/phong/PycharmProjects/Glasses_Detetor/Glasses/5.png: 448x640 1 person, 41.7ms
Speed: 1.5ms preprocess, 41.7ms inference, 9.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/phong/PycharmProjects/Glasses_Detetor/Glasses/4.png: 448x640 1 glasses, 1 sunglasses, 49.9ms
Speed: 1.3ms preprocess, 49.9ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/phong/PycharmProjects/Glasses_Detetor/Glasses/4.png: 448x640 1 person, 6.3ms
Speed: 1.3ms preprocess, 6.3ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/phong/PycharmProjects/Glasses_Detetor/Glasses/9.png: 416x640 1 glasses, 50.8ms
Speed: 1.7ms preprocess, 50.8ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/phong/P